In [1]:
import os
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd 
import numpy as np

In [2]:
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')


In [3]:
con = engine.connect()
engine.table_names()

['Bookings', 'Facilities', 'Members']

In [4]:
# Task 1
# Produce a list of facilities with a total revenue less than 1000.
# The output of facility name and total revenue, sorted by revenue.
rs = con.execute(''' SELECT DISTINCT f.name as facility, CASE WHEN memid != 0 THEN membercost * slots ELSE guestcost * slots END as total_revenue  
    FROM Bookings as b
    JOIN Facilities as f 
    USING (facid)''')

In [5]:
# Retrieve the query's results
q10_df = pd.DataFrame(rs.fetchall())

In [6]:
# Rename columns 
q10_df.columns = ['facility','total_revenue']
# Group the df by facility and sum the values to determine total revenue by facility
tot_rev_by_fac = q10_df.groupby('facility').sum()
# Filter for values less than 1000
tot_rev_by_fac[tot_rev_by_fac['total_revenue'] < 1000]

,total_revenue
facility,
Badminton Court,139.5
Massage Room 2,499.8
Pool Table,15.0
Snooker Table,30.0
Squash Court,637.0
Table Tennis,10.0
Tennis Court 1,840.0
Tennis Court 2,540.0


In [7]:
# Task 2
# Produce a report of members and who recommended them in alphabetic surname,firstname order
rs = con.execute(''' SELECT memid, surname, firstname, recommendedby
    FROM Members
    WHERE memid > 0
    ''')

In [8]:
q11_df = pd.DataFrame(rs.fetchall())
q11_df.columns = ['memid','surname','firstname', 'recommendedby']
q11_df.head(10)

,memid,surname,firstname,recommendedby
0,1,Smith,Darren,
1,2,Smith,Tracy,
2,3,Rownam,Tim,
3,4,Joplette,Janice,1
4,5,Butters,Gerald,1
5,6,Tracy,Burton,
6,7,Dare,Nancy,4
7,8,Boothe,Tim,3
8,9,Stibbons,Ponder,6
9,10,Owen,Charles,1


In [30]:
# Converting memid column to str because recommendedby column type is object
q11_df['memid'] = q11_df['memid'].astype('str')
# Perform self join using merge 
merged_df = q11_df.merge(q11_df, left_on='memid',right_on='recommendedby')
# Clean up values
merged_df['recommenders'] = merged_df['surname_x'] + ", " + merged_df['firstname_x']
merged_df['members'] = merged_df['surname_y'] + ", " + merged_df['firstname_y']
# Final table
result_df = merged_df[['members','recommenders']]
result_df.head()

,members,recommenders
0,"Joplette, Janice","Smith, Darren"
1,"Butters, Gerald","Smith, Darren"
2,"Owen, Charles","Smith, Darren"
3,"Smith, Jack","Smith, Darren"
4,"Mackenzie, Anna","Smith, Darren"


In [9]:
# Task3
# Find the facilities with their usage by member, but not guests
rs = con.execute(''' SELECT f.name as facility, surname, firstname, slots  
FROM Bookings b 
JOIN Facilities f  
USING (facid)
JOIN Members m 
USING (memid)
WHERE memid != 0
    ''')

In [10]:
# Retrieve the query's results
q12_df = pd.DataFrame(rs.fetchall())
# Rename columns
q12_df.columns = ['facility', 'surname', 'firstname', 'slots']
q12_df.head(10)

,facility,surname,firstname,slots
0,Table Tennis,Smith,Darren,2
1,Massage Room 1,Smith,Darren,2
2,Snooker Table,Smith,Darren,2
3,Pool Table,Smith,Darren,1
4,Pool Table,Smith,Darren,1
5,Tennis Court 1,Smith,Tracy,3
6,Tennis Court 1,Smith,Tracy,3
7,Massage Room 1,Rownam,Tim,2
8,Squash Court,Smith,Darren,2
9,Snooker Table,Smith,Tracy,2


In [11]:
q12_df['member'] = q12_df[['surname','firstname']].agg(', '.join, axis=1)
consolidated_df = q12_df.drop(['surname','firstname'],axis=1)
grouped_df = consolidated_df.groupby(['member','facility'])
grouped_df.sum()

slots
member                   facility              
Bader, Florence          Badminton Court     27
                         Massage Room 2       4
                         Pool Table          23
                         Snooker Table       66
                         Squash Court         4
...                                         ...
Tupperware, Hyacinth     Squash Court         2
Worthington-Smyth, Henry Badminton Court     15
                         Massage Room 1       2
                         Pool Table          37
                         Table Tennis         6

[202 rows x 1 columns]

In [12]:
# Task 4 
# Find the facilities usage by month, but not guests
rs = con.execute('''SELECT f.name as facility, slots, starttime
FROM Bookings b 
JOIN Facilities f 
USING (facid)
WHERE memid != 0''')

In [13]:
# Retrieve the query's results
q13_df = pd.DataFrame(rs.fetchall())
q13_df.columns = ['facility', 'slots', 'starttime']
q13_df.head(10)

,facility,slots,starttime
0,Table Tennis,2,2012-07-03 11:00:00
1,Massage Room 1,2,2012-07-03 08:00:00
2,Snooker Table,2,2012-07-03 19:00:00
3,Pool Table,1,2012-07-03 10:00:00
4,Pool Table,1,2012-07-03 15:00:00
5,Tennis Court 1,3,2012-07-04 09:00:00
6,Tennis Court 1,3,2012-07-04 15:00:00
7,Massage Room 1,2,2012-07-04 13:30:00
8,Squash Court,2,2012-07-04 15:30:00
9,Snooker Table,2,2012-07-04 14:00:00


In [33]:
# Extract month from starttime column
q13_df['Month'] = pd.DatetimeIndex(q13_df['starttime']).month
# Group by facility and month and sum for total usage of facilities
grouped_df = q13_df.groupby(['facility','Month']).sum()
grouped_df

slots
facility        Month       
Badminton Court 7        165
                8        414
                9        507
Massage Room 1  7        166
                8        316

In [14]:
# close the connection to db
con.close()